# Single Field Indexes

---
----

### [Single Indexes](https://docs.mongodb.com/manual/core/index-single/)

- Created on a single field.

- Can create in ascending or descending order.


![single index](Images/single_index.svg)

---
### Connecting to MongoDB using Pymongo
----

In [1]:
# Importing the required libraries

import pymongo

import pprint as pp
pp.sorted = lambda x, key=None: x

In [2]:
# Connect to local host

client = pymongo.MongoClient("mongodb://localhost:27017/")

In [3]:
# Connect to database

db = client['nyc']

In [4]:
# Sample document

pp.pprint(
    db.airbnb.find_one()
)

{'_id': ObjectId('60c21bf5b653d40e79b4a7d0'),
 'accom_id': 2595,
 'description': 'Skylit Midtown Castle',
 'host': {'id': 2845,
          'name': 'Jennifer',
          'listings_count': 3,
          'neighbourhood_list': ['Midtown', "Hell's Kitchen"]},
 'neighbourhood': {'name': 'Midtown', 'group': 'Manhattan'},
 'location': {'type': 'Point', 'coordinates': [-73.98559, 40.75356]},
 'room_type': 'Entire home/apt',
 'price': 150,
 'minimum_nights': 30,
 'reviews': {'number_of_reviews': 48,
             'last_review': datetime.datetime(2019, 11, 4, 0, 0),
             'reviews_per_month': 0.35},
 'availability_365': 365}


---
### Querying without index

---

In [5]:
# Query

pp.pprint(
            db.airbnb.find({'accom_id': 2595})\
                     .explain()['executionStats']
        )

{'executionSuccess': True,
 'nReturned': 1,
 'executionTimeMillis': 20,
 'totalKeysExamined': 0,
 'totalDocsExamined': 36905,
 'executionStages': {'stage': 'COLLSCAN',
                     'filter': {'accom_id': {'$eq': 2595}},
                     'nReturned': 1,
                     'executionTimeMillisEstimate': 1,
                     'works': 36907,
                     'advanced': 1,
                     'needTime': 36905,
                     'needYield': 0,
                     'saveState': 36,
                     'restoreState': 36,
                     'isEOF': 1,
                     'direction': 'forward',
                     'docsExamined': 36905},
 'allPlansExecution': []}


---
### Create index using [create_index](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_index)

Create index on `accom_id`.

----

In [6]:
# Create single index

db.airbnb.create_index('accom_id')

'accom_id_1'

In [7]:
# Get indexes

pp.pprint(
    db.airbnb.index_information()
)

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'accom_id_1': {'v': 2, 'key': [('accom_id', 1)]}}


---
### Query using indexed field

Query using indexed field will use the created index instead of `_id` field.

---

In [8]:
# Query

pp.pprint(
            db.airbnb.find({'accom_id': 2595})\
                     .explain()['executionStats']
        )

{'executionSuccess': True,
 'nReturned': 1,
 'executionTimeMillis': 1,
 'totalKeysExamined': 1,
 'totalDocsExamined': 1,
 'executionStages': {'stage': 'FETCH',
                     'nReturned': 1,
                     'executionTimeMillisEstimate': 0,
                     'works': 2,
                     'advanced': 1,
                     'needTime': 0,
                     'needYield': 0,
                     'saveState': 0,
                     'restoreState': 0,
                     'isEOF': 1,
                     'docsExamined': 1,
                     'alreadyHasObj': 0,
                     'inputStage': {'stage': 'IXSCAN',
                                    'nReturned': 1,
                                    'executionTimeMillisEstimate': 0,
                                    'works': 2,
                                    'advanced': 1,
                                    'needTime': 0,
                                    'needYield': 0,
                                    

---
**Naming indexes and changing order of index.**

Providing custom name to an index and creating in `descending order`. By default they are created in ascending order.

----

In [9]:
# Create single index with custom name and un decreasing order

db.airbnb.create_index(
                        [('host.id', pymongo.DESCENDING)],
                        name = 'host_id'
                    )

'host_id'

In [10]:
# Get indexes

pp.pprint(
    db.airbnb.index_information()
)

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'accom_id_1': {'v': 2, 'key': [('accom_id', 1)]},
 'host_id': {'v': 2, 'key': [('host.id', -1)]}}


---
**Cannot create more than one single index with same field.**

----

In [11]:
# Create index

db.airbnb.create_index('accom_id', 
                       name='acommodation_id')

OperationFailure: Index with name: acommodation_id already exists with a different name, full error: {'ok': 0.0, 'errmsg': 'Index with name: acommodation_id already exists with a different name', 'code': 85, 'codeName': 'IndexOptionsConflict'}

---
### Index storage statistics

----

In [12]:
# Index name and size of existing indexes on the collection

db.command('collStats', 'airbnb')['indexSizes']

{'_id_': 344064, 'accom_id_1': 413696, 'host_id': 348160}

In [13]:
# Total size of all indexes

db.command('collStats', 'airbnb')['totalIndexSize']

1105920

----
### Drop indexes

***The disadvantage of creating an index is that it puts a bit of overhead on every insert, update, and delete.***

This is because the database needs to perform the operations and then also make note of it in the indexes of the collection. 

Therefore, it is important to drop indexes in MongoDB which are of no use. This will allow the collection to perform faster writes.



Drop indexes using [drop_index](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.drop_index).

---

In [14]:
# Dropping an index

db.airbnb.drop_index('accom_id_1')

In [15]:
# Get indexes

pp.pprint(
    db.airbnb.index_information()
)

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'host_id': {'v': 2, 'key': [('host.id', -1)]}}


---
Drop all indexes in a collection with [drop_indexes](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.drop_indexes).

---

In [16]:
# Drop all indexes

db.airbnb.drop_indexes()

In [17]:
# Get indexes

db.airbnb.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

---
However, **you can never drop `_id` index.**

---

In [18]:
# Drop index
db.airbnb.drop_index('_id_')

# Get index
db.airbnb.index_information()

OperationFailure: cannot drop _id index, full error: {'ok': 0.0, 'errmsg': 'cannot drop _id index', 'code': 72, 'codeName': 'InvalidOptions'}